## 0. Input files

In [1]:
import os
import operator
import pandas as pd
from Bio.PDB import PDBList
import pandas as pd
import time
import re
import pandas as pd
from itertools import product
import os
import numpy as np
import Bio.PDB as PDB
from Bio.PDB import PDBList, MMCIFParser, Select, Selection, PDBIO, NeighborSearch
from Bio.PDB.MMCIF2Dict import MMCIF2Dict

In [3]:


### Working directory ###
ploop_wdir="/home/servalli/Documents/projects/Ploop_autogenerated"
##subdirs
pdb_dir="PDB"
log_dir="logs"

if not os.path.isdir(ploop_wdir):
    os.mkdir(ploop_wdir)
    
for dir_p in [pdb_dir,log_dir]:
    if not os.path.isdir(os.path.join(ploop_wdir,dir_p)):
        os.mkdir(os.path.join(ploop_wdir,dir_p))


#######PATH DEFINITIONS
ploop_chain_list_p="/home/servalli/Precision/ploop_input/ploop_list.txt" 
#ploop_chain_list_p="/home/servalli/Precision/ploop_input/ploop_list_mini.txt" 

p_dir="/home/servalli/Documents/projects/Ploop_autogenerated/PDB"


pf_dir="ploop_input/pf_ploop_assignment" #!
pth_pdb_mapping="pdbmap"   #from ftp://ftp.ebi.ac.uk/pub/databases/Pfam/current_release/pdbmap.gz
pth_suprfam="ploop_pfam_to_superfam.csv" #manually classified



up_txt="uniprot-pdb.tab" ###uniprot descriptions as table retrieved from up website 13/10/2020
up_mapping="pdb_chain_uniprot.lst" ###NEW MAPPING RETRIEVED 10/10/2020 from ftp://ftp.ebi.ac.uk/pub/databases/msd/sifts/text/pdb_chain_uniprot.lst
pf_mapping="pdb_chain_pfam.lst" ###RETRIEVED 13/10/2020 from ftp://ftp.ebi.ac.uk/pub/databases/msd/sifts/text/pdb_chain_pfam.lst

pf_hmmer="hmmer_pdb_all.txt" ###retrieved from http://www.rcsb.org/pdb/rest/hmmer?file=hmmer_pdb_all.txt

sites_checked="sites_checked_manually.txt"

In [3]:
###Input files ###


ploop_in_dir="/home/servalli/Precision/ploop_input" #folder with all input text files
compound_dir="compound_lists" #folder with lists of PDBs with a given ligand, taken from RCSB PDB (no headers allowed!)
ion_cont_p="MG_MN_SR_CA.txt" #filename for ion presence
#interpro_p="ploop_list.txt" #interpro P-loop structure list, tab-delimited table: 121p	A	IPR027417


In [4]:


interpro_mapping="pdb_chain_interpro.tsv"


#interpro_mapping="/home/servalli/Precision/ploop_input/ploop_list_mini.txt" 
d_ip_mapping=pd.read_csv(os.path.join(pf_dir, interpro_mapping), sep="\t", header=1, names=["pdbid","chain","ipr"])
#print d_up_mapping
ploop_all_chains=d_ip_mapping[d_ip_mapping.ipr=="IPR027417"].copy()
ploop_all_chains['ind_ch']=['_'.join([x, y]) for x, y in zip(ploop_all_chains['pdbid'], ploop_all_chains['chain'])]


In [5]:
#PDBs with ligands lists, taken from RCSB PDB website (9/10/20)

with open(os.path.join(ploop_in_dir,compound_dir,ion_cont_p),"r") as f: has_ion=set(f.readline().strip("\n").split(","))

#with open(os.path.join(ploop_in_dir,interpro_p),"r") as f:   interpro_ids={l.strip("\n").split("\t")[0].upper() for l in f.readlines()}
interpro_ids=set(ploop_all_chains.pdbid.str.upper().to_list())

trinuc_ids=["ATP","ACP","AGS","ANP","GTP","GNP","GSP","GCP"]
compound_pdb={}
for c_id in trinuc_ids:
    with open(os.path.join(ploop_in_dir,compound_dir,f"{c_id}.txt"),"r") as f:
        pdbs=set(f.readline().strip("\n").split(","))
        pdbs_suitable=sorted(list(pdbs&has_ion&interpro_ids))
        compound_pdb[c_id]=pdbs_suitable

gamma_ids=["MGF","ALF","AF3","BEF","VO4"]
gamma_list=[]
for gamma in gamma_ids:
    with open(os.path.join(ploop_in_dir,compound_dir,f"{gamma}.txt"),"r") as f: cur_gamma=f.readline().strip("\n").split(",")
    gamma_list.extend(cur_gamma)
gamma_list=set(gamma_list)
dinuc_ids=["ADP","GDP"]
for c_id in dinuc_ids:
    with open(os.path.join(ploop_in_dir,compound_dir,f"{c_id}.txt"),"r") as f:
        pdbs=set(f.readline().strip("\n").split(","))
        pdbs_suitable=sorted(list(pdbs&has_ion&interpro_ids&gamma_list))
        compound_pdb[c_id]=pdbs_suitable


In [6]:
adps=compound_pdb["ADP"]
gdps=compound_pdb["GDP"]
with open(os.path.join(ploop_in_dir,compound_dir,f"VO4.txt"),"r") as f: vo_gamma=f.readline().strip("\n").split(",")
vo_str=set(vo_gamma).intersection(set(adps).union(set(gdps)))
len(vo_str)
vo_str

{'1DFL',
 '1LKX',
 '1QVI',
 '1VFZ',
 '1VOM',
 '1YV3',
 '2JLX',
 '2V26',
 '3BZ7',
 '3BZ8',
 '3BZ9',
 '3MKD',
 '3MYH',
 '3PUV',
 '4DBR',
 '4E7S',
 '4E7Z',
 '4ZG4',
 '5HMP',
 '5I0I',
 '5N69',
 '5OFR',
 '6I7E'}

## 1. Download structures

In [7]:


def download_pdbs(compound_pdb, ploop_wdir, pdb_dir, log_dir):

    pdbl = PDBList(verbose=False)

    log_p=os.path.join(ploop_wdir,log_dir,"PDB_retrieval.txt")
    curr_time=time.strftime("%Y-%m-%d %H:%M")
    with open(log_p,"w") as log_f:
        log_f.write(f"# 3D structures (mmCif) of P-loop NTPases with NTPs/analogs downloaded, job started at {curr_time} \n")
    colnames=["PDBID","nucl"]
    downloaded=pd.DataFrame(columns=colnames)

    for c_id in compound_pdb.keys():
        success=[]
        save_dir=os.path.join(ploop_wdir,pdb_dir, f"PDB{c_id}","full")
        if not os.path.isdir(save_dir): os.makedirs(save_dir) 
        for i in compound_pdb[c_id]:

            fname=pdbl.retrieve_pdb_file(i,pdir=save_dir, file_format="mmCif")
            fmt=os.path.join(save_dir,f"{i.lower()}.cif")
            if fname==fmt:
                success.append(i)
            else:
                print(fname,i)
        downloaded = pd.concat([downloaded, pd.DataFrame([(i,c_id) for i in success],columns=colnames)])
    with open(log_p,"a") as log_f:
        downloaded.to_csv(log_f,sep="\t",index=False)
    return downloaded

In [8]:
downloaded=download_pdbs(compound_pdb, ploop_wdir, pdb_dir, log_dir)

In [9]:
downloaded

,PDBID,nucl
0,1A82,ATP
1,1D9Z,ATP
2,1DO0,ATP
3,1E2Q,ATP
4,1E79,ATP
...,...,...
84,5NCO,GDP
85,5ND4,GDP
86,5YEW,GDP
87,6B9F,GDP


## 3. Calculate the distances

### new process_pdb_dir

In [10]:



def get_terminals(s,atom,rtype):
    
    all_term_Ns=[re[atom] for re in s.get_residues() if re.resname==rtype and re.has_id(atom) ]
    return all_term_Ns

def get_structinfo(path):
    mmcif_dict = MMCIF2Dict(path)
    
    method=mmcif_dict["_exptl.method"][0].lower()
    pdb_date=mmcif_dict["_pdbx_database_status.recvd_initial_deposition_date"][0] 
    
    if method!="electron microscopy":
        if "_reflns.d_resolution_high" in mmcif_dict:
            resolution=mmcif_dict["_reflns.d_resolution_high"][0]
            if resolution=="?" and "_refine.ls_d_res_high" in mmcif_dict:
                resolution=mmcif_dict["_refine.ls_d_res_high"][0]                
        elif "_refine.ls_d_res_high" in mmcif_dict:
            resolution=mmcif_dict["_refine.ls_d_res_high"][0]
            
            
        else:
            
            resolution=pd.NA
            
       
        
    else:
        resolution=mmcif_dict["_em_3d_reconstruction.resolution"][0]
    if type(resolution)==str:
        if resolution=="?":
            resolution=pd.NA
        else:
            resolution=float(resolution)
    
    p_name=mmcif_dict["_struct.title"][0] 
    
    has_water='HOH' in mmcif_dict["_pdbx_entity_nonpoly.comp_id"]
    
    return method, resolution, p_name, pdb_date, has_water

def process_pdb_dir(p_dir,comps, comps_F, alpha, beta, gamma_F, gamma_N, gamma_types):
    #gia_pdb={comp:(sorted([w for w in os.listdir(os.path.join(p_dir,"PDB"+comp,"full")) if w[3:7] in pdbs]) if os.path.isdir(os.path.join(p_dir,"PDB"+comp,"full")) else []) for comp in comps}
    gia_pdb={comp:(sorted([w for w in os.listdir(os.path.join(p_dir,"PDB"+comp,"full")) if w.endswith(".cif")]) if os.path.isdir(os.path.join(p_dir,"PDB"+comp,"full")) else []) for comp in comps}       
    #Dict with structure filenames
    
    
    asn_cols=[
        "asn_ID","asn_TYPE",
        "asn_ne2-alpha-atom","asn_ne2-alpha-dist","asn_ne2-gamma-atom","asn_ne2-gamma-dist"]

    cols=[
        #protein description
          'superfamily','pfam_acc','pfam_domain','domain_name',
          'Uniprot_Ac','Uniprot_Id',
          'Protein_name_up','Gene_name_up',
        #PDB info
          "PDBID", "model",
        #Nucleotide info
          "nuc_type","nuc_chain","nuc_id",
          "nuc_gamma_moiety_type","nuc_gamma_moiety_chain","nuc_gamma_moiety_id",
        #info on site
          "mg_in_site_4a_from_beta",
          "AG-site","G-site",
          "TYPE_ARG","LYS-site","asn_TYPE",
          
        #Closest non-P-loop Lys
          "Lys_res_ch","Lys_res_id",
          "nz-alpha-atom","nz-alpha-dist","nz-gamma-atom","nz-gamma-dist",
        #Closest Arg 
          
          "Arg_res_ch","Arg_res_id",
          "nh1-alpha-atom","nh1-alpha-dist","nh1-gamma-atom","nh1-gamma-dist",
          "nh2-alpha-atom","nh2-alpha-dist","nh2-gamma-atom","nh2-gamma-dist",
          "ne-alpha-atom","ne-alpha-dist","ne-gamma-atom","ne-gamma-dist",
        #Closest Asn  
        "asn_ID",
        "asn_ne2-alpha-atom","asn_ne2-alpha-dist","asn_ne2-gamma-atom","asn_ne2-gamma-dist",
               
        #Other interactions of gamma-phosphate  with surrounding residues
          
          "Surr_N_BB","Surr_N_SCh",
        #P-loop K-3 backbone to gamma
          "gly13-chain","gly13-type","gly13-id","nuc-to-g13-atom", "dist-gly13",
        #P-loop residue availability
            "lys-ploop-info","ploopk-dist",
        #PDB info
        'resolution',"method","pdbname","pdbdate","include",
        #WB-WA interaction and Mg-coordination
        'SerK+1-Mg','WB-Asp/Glu',"WBD-SerK+1_dist","WBD-Mg","water_present"]
    
    
    dists=pd.DataFrame(columns=cols,index=[0])
    
    df_ind=-1
    ik=0
    epa=MMCIFParser(QUIET=True)
    for ind,comp in enumerate(comps):
        print( comp, len(gia_pdb[comp]), "structures of P-loop NTPases available")
        for T,pdb in enumerate(gia_pdb[comp]):
            if T%50==0:
                print( T, pdb[:4],comp)
            pid=pdb[:4]
            path=os.path.join(p_dir,"PDB"+comp,"full", pdb)
            #print(pdb)
            stru=epa.get_structure(pdb, path)
            ###description

            method, resolution, p_name, pdb_date, has_water=get_structinfo(path)
            if type(resolution)==float:
                if resolution>5:
                    continue
            ###
            
            ###terminals of relevant atoms
            for s in stru.get_models():
                all_term_ND2s_ASN=get_terminals(s,atom="ND2",rtype="ASN")
                all_term_NZs_LYS=get_terminals(s,atom="NZ",rtype="LYS")

                ###ARGS
                all_term_NH1s_ARG=get_terminals(s,atom="NH1",rtype="ARG")
                
                all_term_NH2s_ARG=get_terminals(s,atom="NH2",rtype="ARG")
                
                #negative
                all_term_acid=get_terminals(s,atom="OE1",rtype="GLU")+get_terminals(s,atom="OE2",rtype="GLU")+get_terminals(s,atom="OD1",rtype="ASP")+get_terminals(s,atom="OD2",rtype="ASP")

                ions=[atm for atm in s.get_atoms() if atm.element in ["MG","MN","CA","SR"]]
                
                
                res_all=[res for res in s.get_residues()]
                for ch in s.get_chains():
                    
                    #### Only use chains listed as P-loop NTPases in INTERPRO
                    #### Nucleotides erroneously assigned to activator and not P-loop chain will thus be skipped
                    if pdb[:4]+"_"+ch.get_id() in ploop_all_chains["ind_ch"].values:
                        ##List nucleotides
                        nuc=[res for res in ch.get_residues() if res.get_resname()==comp]
                        ####CHECK FOR INCOMPLETE NUCLEOTIDES AND SKIP THEM
                        nuc_g=[nc for nc in nuc if "O1B" in nc]
                        bad=set(nuc)-set(nuc_g)
                        if len(bad)>0:
                            print("Warning! Truncated nucleotides (no beta-phosphate)", pdb[:4], ch.get_id(), [nu.get_resname()+str(nu.get_id()[1]) for nu in bad])
                        nuc=nuc_g
                        if len(nuc)>0:
                            df_ind=process_nucleotides(dists,
                        s,nuc, comp,df_ind,
                        resolution,method,p_name,pdb_date,has_water, pdb,
                        alpha,beta,gamma_F,gamma_N,comps_F,
                       all_term_NZs_LYS, all_term_NH1s_ARG, all_term_NH2s_ARG, all_term_ND2s_ASN, all_term_acid, ions,
                       asn_cols)
                       
                       


    return dists


### Processing for each nucleotide
* Find gamma-phosphate
* Find Mg2+ or other divalent cation (Ca, Mn, Sr...)
* Find P-loop Lys (K)
---- add here:
    * Find Ser\Thr: K+1, bond to Mg
    * List availability of H2O in structure
    -----* List Mg coordination --- assign W1, W2, W3?
    * Find cat Asp as closest to K+1
    * Measure bond
* Find putative finger residues - list closest Arg, non-P-loop Lys, Asn
* List all protein nitrogens in 4A from gamma

In [12]:
def get_oneletter(three):
    aa_dict={
        "ALA":"A",
        "ASP":"D",
        "ASN":"N",
        "ARG":"R",
        "CYS":"C",
        "GLY":"G",
        "GLU":"E",
        "GLN":"Q",
        "HIS":"H",
        "ILE":"I",
        "LEU":"L",
        "LYS":"K",
        "MET":"M",
        "PRO":"P",
        "PHE":"F",
        "SER":"S",
        "THR":"T",
        "TYR":"Y",
        "TRP":"W",
        "VAL":"V",
        
        
    }
    return aa_dict[three]

In [13]:
not_allowed=["GLU","ASP","SER","THR","TYR","LYS","ARG","HIS"]
[get_oneletter(x) for x in not_allowed]

['E', 'D', 'S', 'T', 'Y', 'K', 'R', 'H']

In [42]:


def get_gamma(s, nc, gamma_types=""):
    beta=["O1B","O2B","O3B","N3B","C3B"]
    alpha=["O1A","O2A","O3A"] 
    gamma=["O1G","O2G","O3G","S1G"]
    gamma_F=["F1","F2","F3","F4"]
    
    if gamma_types=="":
        gamma_types=["ALF","AF3","BEF","MGF","VO4"]
    
    gamma_p=[res for res in s.get_residues() if res.get_resname() in gamma_types]
    
    
    gamma_dict={}
    #ik=ik+1
    #print ik
    dist_g_b=[list((gm, min([atg-nc[atbet] for atbet in beta if atbet in nc]))) for gm in gamma_p for atg in gm.get_atoms()]
    if len(dist_g_b)>0:
        g_cl=min(dist_g_b, key= lambda elem: elem[1])
        #print "Gamma", g_cl, [a for a in g_cl[0].get_atoms()]
        if g_cl[1]<10:
            gamma_dict[nc]=g_cl[0]
    if len(gamma_dict)>0:
        return gamma_dict[nc]
    else:
        return 1


def get_WB_Asp(ch,lys_id,nxt, mg_at,all_term_acid, check_hydro=True,distance_thres=5, recursed=False): 
    #Find Walker B Asp and evaluate Mg binding
    not_allowed=['E', 'D', 'S', 'T', 'Y', 'K', 'R', 'H']
    
    WB_found=False
    hydro=False
    preceding=""
    ser=ch[lys_id+1]
    if "OG1" in ser:
        ser_atom=ser["OG1"]
    else:
        if "OG" in ser:
            ser_atom=ser["OG"]
        else:
            return "SER_NO_TERMINAL_HYDROXYL", "", "","","",""
    
    dst_to_a={ox_at:ser_atom-ox_at for ox_at in all_term_acid}
    
    dst_to_a=dict(sorted(dst_to_a.items(),key=operator.itemgetter(1)))
    
    for atom, dist in dst_to_a.items():
        if WB_found or dist>=distance_thres:
            break
        resn=atom.get_parent()
        rid=resn.get_id()[1]
        #WB_Asp_dist=dist
        aspch=resn.get_parent() #should be the same as ch but who knows as ploop domain is supposed to be one chain 
        #if not all([aspch[rid-x].get_resname() not in not_allowed for x in range (1,4) if rid-x in aspch]):
        preceding="".join([get_oneletter(aspch[rid-x].get_resname()) for x in range (1,4) if rid-x in aspch])
        hydro=not any([x in preceding for x in not_allowed])
        if  hydro or not check_hydro:
            
            WB_found=True
            WB_Asp_dist=dist
            WB_Asp_res=resn
            WB_Asp_info=f"{WB_Asp_res.get_resname()}_{WB_Asp_res.get_parent().get_id()}{WB_Asp_res.get_id()[1]}"
    
            two_terminal_ox=[atom for atom in WB_Asp_res.get_atoms() if atom.name in ["OD1","OD2","OE1","OE2"]]
            if mg_at!="NONE":        
                ser_to_mg=ser_atom-mg_at
        
                WB_to_mg=min([mg_at-ox for ox in two_terminal_ox])             
            else:
                ser_to_mg="NO_MG"
                WB_to_mg="NO_MG" 
        #print (preceding,dist, WB_found)       
    if not WB_found:
        #If we ran out of Asp residues in 5A radius, but could not find WB-Asp, we might be dealing with a protein
        # with a polar substitution in hhhhD/E, such as many Ras-like proteins. We try to rerun search without hydro check:
        if not recursed:
            WB_Asp_info, WB_Asp_dist, ser_to_mg, WB_to_mg, hydro, preceding=get_WB_Asp(ch,lys_id,nxt, mg_at,all_term_acid, check_hydro=False, recursed=True)
        #If we still couldnt find anything, give up and annouce there is no acidic residue around. Probably, open catalytic site.
            if type(WB_Asp_dist)==str:
                WB_Asp_info, WB_to_mg, WB_Asp_dist = "NOT_FOUND", "", ""
        else:
            WB_Asp_info, WB_to_mg, WB_Asp_dist = "NOT_FOUND", "", ""        
        if mg_at!="NONE":        
            ser_to_mg=ser_atom-mg_at
        else:
            ser_to_mg="NO_MG"
    
               
                     
    return WB_Asp_info, WB_Asp_dist, ser_to_mg, WB_to_mg, hydro, preceding
                                
def find_Lys(beta_atoms, all_term_NZs_LYS,all_term_acid, mg_at):
        
        ploop_lys_found=False
        finger_lys_found=False
        ploop_lys=""
        finger_lys=""
        ser_props=["","","","","",""]
        lys_dsts={(NZ,atb):NZ-atb for atb in beta_atoms for NZ in all_term_NZs_LYS}
        if len(lys_dsts)>0:
            lys_dsts={k: v for k, v in sorted(lys_dsts.items(), key=lambda item: item[1])}
            for lysnum, ((NZ,atb), lysdist) in enumerate(lys_dsts.items()):

                lys_id=NZ.get_parent().get_id()[1]                    
                ch=NZ.get_parent().get_parent()
                if not ploop_lys_found:
                    
                    if ch.has_id(lys_id+1): #If next residue number exist in the structure
                        nxt=ch[lys_id+1].get_resname()
                        if nxt=="SER" or nxt=="THR":
                            ploop_lys=NZ.get_parent()
                            ser_props=get_WB_Asp(ch,lys_id,nxt, mg_at,all_term_acid)
                            
                            ploop_lys_found=True

                        else:
                            finger_lys=NZ.get_parent()
                            finger_lys_found=True

                    else:
                        finger_lys=NZ.get_parent()
                        finger_lys_found=True 

                else:
                    if not finger_lys_found and (lys_id!=ploop_lys.get_id()[1] or ch.get_id()!=ploop_lys.get_parent().get_id()) :
                        finger_lys=NZ.get_parent()
                        finger_lys_found=True

                if finger_lys_found and ploop_lys_found:
                    break
        
        return ploop_lys, finger_lys, ser_props
    
def calc_Asn(alpha_atoms, beta_atoms, gamma_atoms,all_term_ND2s_ASN):
    asn_dsts={(NE2,atb):NE2-atb for atb in beta_atoms for NE2 in all_term_ND2s_ASN}
    if len(asn_dsts)>0:
        
            asn_closest_ne,atb_cl=min(asn_dsts, key=asn_dsts.get)
            
            dst_to_a={at:asn_closest_ne-at for at in alpha_atoms}
            dst_to_g={at: asn_closest_ne-at for at in gamma_atoms}           
            
            a_closest=min(dst_to_a, key=dst_to_a.get)
            g_closest=min(dst_to_g, key=dst_to_g.get)
            
            #asn_cols=["asn_ID","asn_TYPE","asn_ne2-alpha-atom","asn_ne2-alpha-dist","asn_ne2-gamma-atom","asn_ne2-gamma-dist"]

            asn_line=f"{asn_closest_ne.get_parent().get_parent().get_id()}{asn_closest_ne.get_parent().get_id()[1]}","",a_closest.id,dst_to_a[a_closest],g_closest.id,dst_to_g[g_closest]
            return asn_line,asn_closest_ne.get_parent()
    else:
        return "",""


def calc_ag_distances(residue,atom_name, alpha_atoms,gamma_atoms):
    """
    Calculate shortest distances from <atom_name> atom of <residue> (Biopython PDB Residue object)
    to alpha-phosphate oxygens (supplied as Biopython PDB Atom object list <alpha_atoms>)
    and to gamma-phosphate oxygens (idem, as <gamma_atoms>)
    """
    #if len(gamma_atoms)>0:
    if residue.has_id(atom_name):
        alpha_atom,alpha_dist=min([(at,residue[atom_name]-at) for at in alpha_atoms], key=lambda x: x[1])     
        gamma_atom,gamma_dist=min([(at,residue[atom_name]-at) for at in gamma_atoms], key=lambda x: x[1])

        return alpha_atom.get_name(),alpha_dist,gamma_atom.get_name(),gamma_dist
    else: return "","","",""
    #else:
    #    print(residue, gamma_atoms)
    

def list_all_Ns(gamma_atoms, distlimit, excluded_sidechain, excluded_backbone):
    """
    Return two lists of tuples with information on backbone and sidechain nitrogens interacting
    (located closer than <distlimit> Angstrom) with gamma-phosphate oxygens <gamma_atoms>,
    excluding residues provided in lists <excluded_sidechain>, <excluded_backbone>.
    Output lists consist of tuples (interacting_nitrogen_atom,interacting_gamma_atom, distance)
    """
    nc=gamma_atoms[0].get_parent()
    model = nc.get_parent().get_parent()

    atom_list = [atom for atom in model.get_atoms() if atom.element == 'N' and atom.get_parent()!=nc]
    backbone_list= [atom for atom in atom_list if atom.get_name()=="N" and atom.get_parent() not in excluded_backbone]
    if len(backbone_list)==0:
        return "",""
    sidechain_list=[atom for atom in atom_list if atom.get_name()!="N" and atom.get_parent() not in excluded_sidechain]
    ns_bb = NeighborSearch(backbone_list)
    ns_sch= NeighborSearch(sidechain_list)

    nearby_nitro_bb = [(atm,ga,atm-ga) for ga in gamma_atoms for atm in ns_bb.search(ga.coord, distlimit, 'A')]
    nearby_nitro_sch = [(atm,ga,atm-ga) for ga in gamma_atoms for atm in ns_sch.search(ga.coord, distlimit, 'A')]
    sidechains={}
    for atm,ga,dist in nearby_nitro_sch:
        if atm.get_parent() in sidechains:
            sidechains[atm.get_parent()].append((atm,ga,dist))
        else:
            sidechains[atm.get_parent()]=[(atm,ga,dist)]
        

    return nearby_nitro_bb, sidechains

    
def process_nucleotides(dists,
                        s,nuc, comp,df_ind,
                        resolution,method,p_name,pdbdate,has_water, pdb,
                        alpha,beta,gamma_F,gamma_N,comps_F,
                       all_term_NZs_LYS, all_term_NH1s_ARG, all_term_NH2s_ARG, all_term_ND2s_ASN, all_term_acid, ions,
                       asn_cols
                       
                       ):

    ##Calculate the dists for each chain
    for nc in nuc:
        if comp in comps_F:
            gamma=get_gamma(s, nc)
            if type(gamma)==int:
                print(f"Gamma not found! {pdb[:4]} {nc.get_resname()} {nc.get_parent().get_id()}{nc.get_id()[1]}, skipping")
                continue
            gamma_atoms=[at for at in gamma.get_atoms() if at.name in gamma_F]
                
        else:
            gamma_atoms=[at for at in nc.get_atoms() if at.name in gamma_N]
            if len(gamma_atoms)==0:
                print(f"Gamma not found! {pdb[:4]} {nc.get_resname()} {nc.get_parent().get_id()}{nc.get_id()[1]}, skipping")
                continue


        beta_atoms=[at for at in nc.get_atoms() if at.name in beta] 
        alpha_atoms=[at for at in nc.get_atoms() if at.name in alpha]
        
        ##Check MG ion in 5A radius from beta phopsphates
        ion_dsts={(M,atb):M-atb for atb in beta_atoms for M in ions}
        if len(ion_dsts)>0:        
            mg_at,atb_cl=min(ion_dsts, key=ion_dsts.get)
            if mg_at-atb_cl>5: mg_at="NONE" 
        else:
            mg_at="NONE"
                     
        if mg_at!="NONE":
            mg_at_e=mg_at.element
        else:
            mg_at_e="NONE"
            
        #################
        df_ind=df_ind+1
        ################
        
        #Write Mg search results
        dists.loc[df_ind,"mg_in_site_4a_from_beta"]=mg_at_e #this should create the line
        dists.loc[df_ind,"water_present"]=has_water
        
        
        #Write General struct props
        dists.loc[df_ind,['resolution',"method","pdbname","pdbdate","PDBID"]]=resolution,method,p_name,pdbdate,pdb[:4]

        #write Nucleotide info
        
        dists.loc[df_ind,["nuc_type","nuc_chain","nuc_id","model"]]=comp,nc.get_parent().get_id(),nc.get_id()[1],nc.get_parent().get_parent().get_id()

        #write info on gamma-mimic, if any
                  
        if comp in comps_F:
            dists.loc[df_ind,["nuc_gamma_moiety_type","nuc_gamma_moiety_chain","nuc_gamma_moiety_id"]]=gamma.get_resname(),gamma.get_parent().get_id(),gamma.get_id()[1]        
  
                  
        #####LYS
        ploop_lys, finger_lys,ser_props=find_Lys(beta_atoms, all_term_NZs_LYS,all_term_acid, mg_at) 
        
        
        
        dists.loc[df_ind,['WB-Asp/Glu',"WBD-SerK+1_dist",'SerK+1-Mg',"WBD-Mg","is_hydro","preceding_res"]]=ser_props

        #P-loop Lys
        if ploop_lys!="":          
            ploop_ch=ploop_lys.get_parent().get_id() #It should be the same as nucleotide chain. But in real PDBs...
            ploop_id=ploop_lys.get_id()[1]   
            ploop_dist=min([ploop_lys["NZ"]-atb for atb in beta_atoms])
            dists.loc[df_ind,["lys-ploop-info","ploopk-dist"]]= f"{ploop_ch}{ploop_id}",ploop_dist 
        #Finger Lys
        if finger_lys!="":
            Kdist=calc_ag_distances(finger_lys,"NZ", alpha_atoms,gamma_atoms)         
            dists.loc[df_ind,["Lys_res_ch","Lys_res_id",
                              "nz-alpha-atom","nz-alpha-dist",
                              "nz-gamma-atom","nz-gamma-dist"
                             ]]= finger_lys.get_parent().get_id(),finger_lys.get_id()[1],*Kdist        
                 

        
        #####ASN
        asn_line,finger_asn=calc_Asn(alpha_atoms, beta_atoms, gamma_atoms,all_term_ND2s_ASN)
        if asn_line!="":
            dists.loc[df_ind,asn_cols]=asn_line
        
        
        ####ARG
        if len(all_term_NH1s_ARG)>0:


            arg_dsts={(NH,atb):NH-atb for atb in beta_atoms for NH in all_term_NH1s_ARG+all_term_NH2s_ARG}
            

            arg_closest_nhx,atb_cl=min(arg_dsts, key=arg_dsts.get)
            finger_arg=arg_closest_nhx.get_parent()
            dists.loc[df_ind,["Arg_res_ch","Arg_res_id"]]=finger_arg.get_parent().get_id(),finger_arg.get_id()[1]

            #NH1
            Rdist1=calc_ag_distances(finger_arg,"NH1", alpha_atoms,gamma_atoms)     
            dists.loc[df_ind,["nh1-alpha-atom","nh1-alpha-dist","nh1-gamma-atom","nh1-gamma-dist"]]=Rdist1          

            #NH2
            Rdist2=calc_ag_distances(finger_arg,"NH2", alpha_atoms,gamma_atoms)
            dists.loc[df_ind,["nh2-alpha-atom","nh2-alpha-dist","nh2-gamma-atom","nh2-gamma-dist"]]=Rdist2 

            #NE          
            Rdiste=calc_ag_distances(finger_arg,"NE", alpha_atoms,gamma_atoms)
            dists.loc[df_ind,["ne-alpha-atom","ne-alpha-dist","ne-gamma-atom","ne-gamma-dist"]]=Rdiste           
        
        else: finger_arg=""
        #####K-3 residue
        if ploop_lys!="":
            gly13=ploop_lys.get_parent()[ploop_id-3]
            nuctog13a,g13_dist=min([(at,gly13["N"]-at) for at in gamma_atoms], key=lambda x: x[1])          
            dists.loc[df_ind,["gly13-chain","gly13-type","gly13-id",
                              "nuc-to-g13-atom", "dist-gly13"]]=ploop_ch,gly13.get_resname(),ploop_id-3,nuctog13a.get_name(),g13_dist
            excluded_bb=[gly13]
        else:
            excluded_bb=[]
       
 
        ####Additional interactions with other residues

        excluded_sch=[res for res in [ploop_lys,finger_lys,finger_arg,finger_asn] if type(res)!=str]
               
        bb,sch=list_all_Ns(gamma_atoms, 4, excluded_sch,excluded_bb)

        if len(bb)!=0:
            ln_bb="; ".join([f"{N.get_parent().get_resname()}_{N.get_parent().get_parent().get_id()}{N.get_parent().get_id()[1]}..{Og.get_name()}({dist:.2f})" for N,Og,dist in bb])
        else:
            if bb=="":
                print(f"Warning! cannot find backbone nitrogens. {pdb[:4]} {nc.get_resname()} {nc.get_parent().get_id()}{nc.get_id()[1]}")
            ln_bb=""
        
        if len(sch)>0:            
            #ln_sch="; ".join([f"{N.get_parent().get_resname()}_{N.get_parent().get_parent().get_id()}{N.get_parent().get_id()[1]}_{N.get_name()}..{Og.get_name()}({dist:.2f})" for N,Og,dist in sch])          
            sep2=","
            cont_temp='{}..{}({:.2f})'
            ln_sch="; ".join([f"{res.get_resname()}_{res.get_parent().get_id()}{res.get_id()[1]}_{sep2.join([cont_temp.format(N.get_name(),Og.get_name(),dist) for (N,Og,dist) in valuelist if (float(dist)<=3.2 or dist==min(valuelist, key=lambda x:float(x[2]))[2])])}" for res,valuelist in sch.items()])          
            
        else:
            ln_sch=""
        dists.loc[df_ind,["Surr_N_BB","Surr_N_SCh"]]=ln_bb,ln_sch
    return df_ind 


### Functions for table processing

In [15]:
def mark_repeated_nmr(dists):
    #inplace
    for model in range(1,max(dists.model)):
        nmrs=dists[~dists["include"].isin(["NO_PLOOP","KPLDIST>5","NOMG","NMR_RED"])&(dists["method"]=="solution nmr")&(dists["model"]==model)]
        #print(model)
        #display(nmrs)
        if nmrs.shape[0]>0:
            for indr, row in nmrs.iterrows():
                p,n,i=row[["PDBID","nuc_chain","nuc_id"]]
                #print(p,n,i)
                models_bef=dists[(dists["model"]<model)&(dists["PDBID"]==p)&(dists["nuc_chain"]==n)&(dists["nuc_id"]==i)]
                if models_bef.shape[0]>0:
                    dists.loc[(dists["PDBID"]==p)&(dists["nuc_chain"]==n)&(dists["nuc_id"]==i)&(~dists["include"].isin(["NO_PLOOP","KPLDIST>5","NOMG"]))&(dists["method"]=="solution nmr")&(dists["model"]>=model),"include"]="NMR_RED"
                    
                    
def mark_lowqual_sites(dists):
    #inplace
    dists.loc[(dists["ploopk-dist"].isna()),"include"]="NO_PLOOP_R"
    dists.loc[(dists["ploopk-dist"]>5)&(dists["include"].isna()),"include"]="KPLDIST>5"
    dists.loc[(dists["mg_in_site_4a_from_beta"]=="NONE")&(dists["include"].isna()),"include"]="NOMG"
    dists.loc[dists.resolution>5,"include"]="BAD_RES"
    dists.loc[((dists["nh1-alpha-dist"]=="")|(dists["nh2-alpha-dist"]==""))&dists["include"].isna(),"include"]="FAULTY_ARG"
    ###Closest Arg residue lacks one of the terminal NH2s, this will produce errors later
    mark_repeated_nmr(dists)
                    
def assign_uniprot(dists,pdb,chain,row, d_up_mapping,d_up_info, indexr):
        ids_m=d_up_mapping.loc[(d_up_mapping['PDB']==pdb)&(d_up_mapping['CHAIN']==chain)]
        if not pd.isnull(chain):
            ploop_n=row["gly13-id"]+3
        else:
            ploop_n=pd.nan
        upid="None"    
        if ids_m.shape[0]>0: 
            if ids_m.shape[0]>1 : #If several ids are mapped to this chain and ploop  region is known, get the right id
                if ploop_n!="":
                    if ids_m.loc[(ids_m.RES_BEG<=ploop_n)&(ids_m.RES_END>=ploop_n),'SP_PRIMARY'].shape[0]>0:
                        upid=ids_m.loc[(ids_m.RES_BEG<=ploop_n)&(ids_m.RES_END>=ploop_n),'SP_PRIMARY'].values[0]

            if upid=="None":    
                upid=ids_m.SP_PRIMARY.values[0]
                #print( upid)
        
            

        if upid!="None":
            upinfo=d_up_info.loc[d_up_info['Entry'].values==upid,:]
            if upinfo.shape[0]>0:
                uname=upinfo['Entry name'].values[0]
                p_longname=upinfo['Protein names'].values[0]
                pname=re.split('[\(\)\[\]]',p_longname)[0]
                gname=upinfo['Gene names'].values[0]
                pfams=upinfo["Cross-reference (Pfam)"].values[0]


                dists.loc[indexr,['Uniprot_Ac','Uniprot_Id','Protein_name_up','Gene_name_up']]=upid,uname,pname,gname
        else:
            print("No Uniprot ID, ", pdb, chain)
        return upid,ploop_n

def get_pfamid(pdb, chain, ploop_n, d_pfam_hmm, pfams_listed_as_pl):
    pf_i=d_pfam_hmm[(d_pfam_hmm.PDB_ID==pdb.upper())&(d_pfam_hmm.CHAIN_ID==chain)]
    pfam=""

    if pf_i.shape[0]>0:
        pfams=set(pf_i.PFAM.to_list()).intersection(pfams_listed_as_pl)
        if not pd.isnull(ploop_n):

            pf_bycoord=pf_i[(pf_i.start<=ploop_n)&(pf_i.end>=ploop_n)&pf_i.PFAM.isin(pfams)]

            #Pfam df was already sorted by value, so then
            if pf_bycoord.shape[0]>0:
                pfam, pfam_n, pfam_descr=pf_bycoord[["PFAM","PFAM_Name","PFAM_desc"]].values[0]
                comment="Ok"
        else:
            
            if len(pfams)>0:
                pfam, pfam_n, pfam_descr=pf_i.loc[pf_i.PFAM.isin(pfams),["PFAM","PFAM_Name","PFAM_desc"]].values[0]               

                comment="coordinate missmatch,picking best P-loop dom"
            
    if pfam=="": 
        pfam, pfam_n, pfam_descr, comment="None","None","None","not found"
            
    return pfam, pfam_n, pfam_descr, comment    

def add_identifiers(dists, d_ip_mapping, d_up_info, d_pfam_hmm, d_pfam_mapping, pf_to_suprf):
    for indexr, row in dists.iterrows():

        pdb=row['PDBID']
        chain=row['gly13-chain']
        if pd.isnull(chain):
            chain=row['nuc_chain']
        #print pdb
        #if pdb in all_pdbs_mapped:
        upid,ploop_n=assign_uniprot(dists,pdb,chain,row, d_up_mapping,d_up_info, indexr)
        pfam=""
        pfam, pfam_n, pfam_descr, pfc=get_pfamid(pdb, chain, ploop_n, d_pfam_hmm, pfams_listed_as_pl)


        if pfam=="None":
            #Check if other chains with similar sequence were mapped
            """pf_alt_chain=d_pfam_hmm[(d_pfam_hmm.PDB_ID==pdb.upper())].CHAIN_ID.to_list()
            if len(pf_alt_chain)>0:
                sim_site=dists[(dists.PDBID==pdb)&(dists["gly13-chain"].isin(pf_alt_chain))&(dists["gly13-id"]==row["gly13-id"])]
                if sim_site.shape[0]>0:
                    chain_similar=sim_site["gly13-chain"].values[0]
                    pfam, pfam_n, pfam_descr, pfc=get_pfamid(pdb, chain_similar, ploop_n, d_pfam_hmm, pfams_listed_as_pl)

                """
            pf_alt_chain=d_pfam_hmm[(d_pfam_hmm.PDB_ID==pdb.upper())].CHAIN_ID.to_list()
            alt_chains=d_pfam_mapping[(d_pfam_mapping.PDB==pdb)&(d_pfam_mapping.SP_PRIMARY==upid)].CHAIN.to_list()
            alt_chains.extend(pf_alt_chain)
            if len(alt_chains)>0:
                sim_site=dists[(dists.PDBID==pdb)&(dists["gly13-chain"].isin(alt_chains))&(dists["gly13-id"]==row["gly13-id"])]
                if sim_site.shape[0]>0:
                    chain_similar=sim_site["gly13-chain"].values[0]
                    pfam, pfam_n, pfam_descr, pfc=get_pfamid(pdb, chain_similar, ploop_n, d_pfam_hmm, pfams_listed_as_pl)
                """display(chain_similar, sim_site, chain)

                alt_chains=set(d_pfam_mapping[(d_pfam_mapping.PDB==pdb)&(d_pfam_mapping.SP_PRIMARY==upid)].CHAIN.to_list())
                chains_same=list((alt_chains-set(chain)).intersection(set(pf_alt_chain)))
                if len(chains_same)!=0:
                    pfam, pfam_n, pfam_descr, pfc=get_pfamid(pdb, chains_same[0], ploop_n, d_pfam_hmm, pfams_listed_as_pl)

                else:
                    if len(alt_chains)>0:
                        pfams=d_pfam_mapping[(d_pfam_mapping.PDB==pdb)&(d_pfam_mapping.SP_PRIMARY==upid)].PFAM_ID.values
                        if len(pfams_listed_as_pl.intersection(pfams))>0:
                            pfam=list(pfams_listed_as_pl.intersection(pfams))[0]
                            pfam_n,pfam_descr=d_pfam_hmm.loc[d_pfam_hmm.PFAM==pfam,["PFAM_Name","PFAM_desc"]].values[0]
                            pfc="assigned without coord check"""



        if pfam=="None":

            pfams=d_pfam_mapping[(d_pfam_mapping.SP_PRIMARY==upid)].PFAM_ID.values
            if len(pfams_listed_as_pl.intersection(pfams))>0:
                pfam=list(pfams_listed_as_pl.intersection(pfams))[0]
                pfam_n,pfam_descr=d_pfam_hmm.loc[d_pfam_hmm.PFAM==pfam,["PFAM_Name","PFAM_desc"]].values[0]
                pfc="assigned by UP id"


        if pfam=="None":
            if pdbs_ploop.loc[(pdbs_ploop['pdb_id']==pdb)&(pdbs_ploop['CHAIN_ID']==chain)].shape[0]>0:
                pfam=pdbs_ploop.loc[(pdbs_ploop['pdb_id']==pdb)&(pdbs_ploop['CHAIN_ID']==chain)]["PFAM_ACC"].values[0]
                pfam_n,pfam_descr=d_pfam_hmm.loc[d_pfam_hmm.PFAM==pfam,["PFAM_Name","PFAM_desc"]].values[0]

                pfc="assigned from pdbmap"
        dists.loc[indexr,["pfam_acc",'pfam_domain','domain_name',"pfam_comm"]]=pfam, pfam_n, pfam_descr,pfc        
        if pfam in pfams_listed_as_pl:
            sfam=pf_to_suprf.loc[pf_to_suprf['pf_id']==pfam,"superfamily"].values[0]

            dists.loc[indexr,["superfamily"]]=sfam
        else:
            if f"{pdb}_{chain}" in all_pdbs_ploop:
                dists.loc[indexr,["superfamily"]]="Unassigned P-loop (PFAM not mapped)"

## BODY - definitions for distance calculations

In [16]:



comps_F=["GDP","ADP"]
comps=["ADP","ATP","ANP","ACP","AGS","GNP","GCP","GSP","GTP","GDP"]

beta=["O1B","O2B","O3B","N3B","C3B"]
alpha=["O1A","O2A","O3A"] 
gamma_N=["O1G","O2G","O3G","S1G"]
gamma_F=["F1","F2","F3","F4","O1","O2","O3","O4"]

gamma_types=["ALF","AF3","BEF","MGF","VO4"]


#####
pd.set_option('display.max_columns', 1000)
#####






### Read tables for annotation

In [17]:
d_pfam_mapping=pd.read_csv(os.path.join(pf_dir, pf_mapping), sep="\t", header=1)

d_pfam_hmm=pd.read_csv(os.path.join(pf_dir, pf_hmmer), sep="\t", header=0)


d_pfam_hmm["PFAM"]=d_pfam_hmm.PFAM_ACC.str.split(".",expand=True)[0]
d_pfam_hmm[["start","end"]]=-10000000,10000000
isnumeric_start=d_pfam_hmm.PdbResNumStart.str.lstrip("-").str.isdigit()
isnumeric_end=d_pfam_hmm.PdbResNumEnd.str.lstrip("-").str.isdigit()
d_pfam_hmm.loc[isnumeric_start,"start"]=d_pfam_hmm.loc[isnumeric_start,"PdbResNumStart"].astype(int)
d_pfam_hmm.loc[isnumeric_end,"end"]=d_pfam_hmm.loc[isnumeric_end,"PdbResNumEnd"].astype(int)
d_pfam_hmm.sort_values(by="eValue", inplace=True)


col_ns_pdb_pf=['PDB_ID','CHAIN_ID',2,"PFAM_Name", "PFAM_ACC",	"Accesion", "Position"]
pf_to_pdb = pd.read_csv(os.path.join(pf_dir, pth_pdb_mapping), sep="\t", header=None, names=col_ns_pdb_pf)
display(pf_to_pdb)
for col in pf_to_pdb:
	if col!=2:
		print( col)
		pf_to_pdb[col] = pf_to_pdb[col].map(lambda x: x.strip(';'))
		
	
pf_to_suprf=pd.read_csv(os.path.join(pf_dir, pth_suprfam), sep=",", header=0)

accesions=list(pf_to_suprf['pf_id'])
pdbs_ploop=pf_to_pdb.loc[pf_to_pdb["PFAM_ACC"].isin(accesions)].copy()
pdbs_ploop['pdb_id']=pdbs_ploop['PDB_ID'].str.lower()

pdb_to_up={}


d_up_info=pd.read_csv(os.path.join(pf_dir, up_txt), sep="\t", header=0)
#print d_up_info
d_up_mapping=pd.read_csv(os.path.join(pf_dir, up_mapping), sep="\t", header=1)
#print d_up_mapping
d_pfam_mapping=pd.read_csv(os.path.join(pf_dir, pf_mapping), sep="\t", header=1)

all_pdbs_ploop=set(ploop_all_chains.ind_ch.to_list())
pfams_listed_as_pl=set(pf_to_suprf.pf_id.to_list())


,PDB_ID,CHAIN_ID,2,PFAM_Name,PFAM_ACC,Accesion,Position
0,4ZLP;,A;,NaN,Notch;,PF00066;,Q9UM47;,1385-1418;
1,5CZX;,B;,NaN,Notch;,PF00066;,Q9UM47;,1387-1418;
2,5CZV;,A;,NaN,Notch;,PF00066;,Q9UM47;,1391-1418;
3,5CZX;,A;,NaN,Notch;,PF00066;,Q9UM47;,1387-1418;
4,4ZLP;,B;,NaN,Notch;,PF00066;,Q9UM47;,1392-1418;
...,...,...,...,...,...,...,...
698484,1GM5;,A;,NaN,RecG_C;,PF19833;,Q9WY48;,711-755;
698485,6OX6;,A;,NaN,Ntox46;,PF15538;,Q9I739;,266-373;
698486,4ZV4;,D;,NaN,Ntox46;,PF15538;,Q9I739;,266-422;
698487,4ZV0;,A;,NaN,Ntox46;,PF15538;,Q9I739;,282-422;


PDB_ID
CHAIN_ID
PFAM_Name
PFAM_ACC
Accesion
Position


In [18]:
pf_to_pdb


,PDB_ID,CHAIN_ID,2,PFAM_Name,PFAM_ACC,Accesion,Position
0,4ZLP,A,NaN,Notch,PF00066,Q9UM47,1385-1418
1,5CZX,B,NaN,Notch,PF00066,Q9UM47,1387-1418
2,5CZV,A,NaN,Notch,PF00066,Q9UM47,1391-1418
3,5CZX,A,NaN,Notch,PF00066,Q9UM47,1387-1418
4,4ZLP,B,NaN,Notch,PF00066,Q9UM47,1392-1418
...,...,...,...,...,...,...,...
698484,1GM5,A,NaN,RecG_C,PF19833,Q9WY48,711-755
698485,6OX6,A,NaN,Ntox46,PF15538,Q9I739,266-373
698486,4ZV4,D,NaN,Ntox46,PF15538,Q9I739,266-422
698487,4ZV0,A,NaN,Ntox46,PF15538,Q9I739,282-422


### Calculate distances,
### Filter site table and add protein descriptions





In [43]:
year=2022
spec="Ver5_hy_fallback"

dists=process_pdb_dir(p_dir,comps, comps_F, alpha, beta,  gamma_F, gamma_N, gamma_types)
dists_raw=dists.copy()
dists.to_csv(os.path.join(ploop_wdir,f"{year}_{spec}_t1_raw_dist.tsv"),sep="\t")
mark_lowqual_sites(dists)
add_identifiers(dists, d_ip_mapping, d_up_info, d_pfam_hmm, d_pfam_mapping, pf_to_suprf)
dists.to_csv(os.path.join(ploop_wdir,f"{year}_{spec}_t2_with_family.tsv"),sep="\t")
dists2=dists.copy()
dists_selected=dists[dists.include.isna()].copy()

ADP 144 structures of P-loop NTPases available
0 1br1 ADP
Gamma not found! 1h8e ADP A600, skipping
Gamma not found! 1h8e ADP B600, skipping
Gamma not found! 1h8e ADP C600, skipping
Gamma not found! 1h8e ADP E600, skipping
Gamma not found! 1ihu ADP A590, skipping
Gamma not found! 1vfz ADP A500, skipping
Gamma not found! 1w0j ADP A1511, skipping
Gamma not found! 1w0j ADP B1511, skipping
Gamma not found! 1w0j ADP C1511, skipping
50 3kql ADP
100 5lta ADP
Gamma not found! 6ap1 ADP D701, skipping
Gamma not found! 6ap1 ADP E701, skipping
Gamma not found! 6bmf ADP D501, skipping
Gamma not found! 6bmf ADP E501, skipping
Gamma not found! 6gej ADP T501, skipping
Gamma not found! 6gej ADP U502, skipping
Gamma not found! 6gej ADP V501, skipping
Gamma not found! 6gej ADP W501, skipping
Gamma not found! 6gej ADP X501, skipping
Gamma not found! 6gej ADP Y501, skipping
Gamma not found! 6gen ADP T501, skipping
Gamma not found! 6gen ADP U502, skipping
Gamma not found! 6gen ADP V501, skipping
Gamma not fo

In [84]:

dists[dists["WB-Asp/Glu"]!="NOT_FOUND"].shape

                      


(1963, 67)

### Functions for site configuration

In [30]:
def assign_arg_type(dists, d_sites_checked=""):
    distance_columns=[col for col in dists.columns if "dist" in col and col!="WBD-SerK+1_dist"]
    for col in distance_columns:
        dists[col]=dists[col].astype(float)
    #notype=(dists["TYPE_ARG"].isna())&(dists["include"].isna())

    dists.loc[(((dists["nh2-gamma-dist"]<4)&(dists["nh1-alpha-dist"]<4))|((dists["nh1-gamma-dist"]<4)&(dists["nh2-alpha-dist"]<4))),"TYPE_ARG"]="FORK."
    dists.loc[(dists["nh1-gamma-dist"]<4)&(dists["nh1-alpha-dist"]<4),"TYPE_ARG"]="NH1 weak."
    dists.loc[(dists["nh1-gamma-dist"]<3.2)&(dists["nh1-alpha-dist"]<3.2),"TYPE_ARG"]="NH1."
    dists.loc[(dists["nh2-gamma-dist"]<4)&(dists["nh2-alpha-dist"]<4),"TYPE_ARG"]="NH2 weak."
    dists.loc[(dists["nh2-gamma-dist"]<3.2)&(dists["nh2-alpha-dist"]<3.2),"TYPE_ARG"]="NH2."
    dists.loc[(((dists["nh2-gamma-dist"]<4)|(dists["nh1-gamma-dist"]<4))&((dists["nh1-alpha-dist"]>4)&(dists["nh2-alpha-dist"]>4))),"TYPE_ARG"]="ONLY GAMMA weak."
    dists.loc[(((dists["nh2-gamma-dist"]<3.2)|(dists["nh1-gamma-dist"]<3.2))&((dists["nh1-alpha-dist"]>4)&(dists["nh2-alpha-dist"]>4))),"TYPE_ARG"]="ONLY GAMMA."
    dists.loc[(((dists["nh2-gamma-dist"]>4)&(dists["nh1-gamma-dist"]>4))),"TYPE_ARG"]="NONE."
    if type(d_sites_checked)!=str:
        for ind,row in d_sites_checked.iterrows():
            dists.loc[(dists.PDBID==row.PDB)&((dists.nuc_chain+dists.nuc_id.astype(str))==row.nuc),"TYPE_ARG"]=row.argtype


In [31]:
def assign_mono_type(dists):
    dists.loc[(dists["nz-gamma-dist"]<4),"LYS-site"]="G"
    dists.loc[(dists["nz-gamma-dist"]<4)&(dists["nz-alpha-dist"]<4),"LYS-site"]="AG"

    dists.loc[(dists["LYS-site"].isna()),"LYS-site"]="NONE"
    
    
    dists.loc[(dists["asn_ne2-gamma-dist"]<4),"asn_TYPE"]="(only G)"
    dists.loc[(dists["asn_ne2-gamma-dist"]<4)&(dists["asn_ne2-alpha-dist"]<4),"asn_TYPE"]="AG_weak"
    dists.loc[(dists["asn_ne2-gamma-dist"]<3.2)&(dists["asn_ne2-alpha-dist"]<3.2),"asn_TYPE"]="AG"
    dists.loc[(dists["asn_ne2-gamma-dist"]>4),"asn_TYPE"]="NONE"
    dists.loc[(dists["asn_ne2-gamma-dist"].isna()),"asn_TYPE"]="NO_ASN"



In [33]:
def assign_type(dists):
    

    
    dists.loc[dists["LYS-site"]=="AG","AG-site"]="LYS"
    dists.loc[dists.TYPE_ARG.str.contains("NH"),"AG-site"]="ARG"
    dists.loc[dists.TYPE_ARG.str.contains("FORK"),"AG-site"]="ARG"
    dists.loc[dists.asn_TYPE.str.contains("AG"),"AG-site"]="ASN"
    dists.loc[dists["AG-site"].isna(),"AG-site"]="NONE"
    
    #GAMMA
    g_main_k_g=(dists["LYS-site"]=="G")
    
    g_main_r_g=dists.TYPE_ARG.str.contains("GAMMA")
    

    
    n_lys_g=dists["Surr_N_SCh"].str.count("LYS")+g_main_k_g.astype(int)   
    
    n_arg_g=dists["Surr_N_SCh"].str.count("ARG")+g_main_r_g.astype(int)
    
    comma=(np.sign(n_arg_g)*np.sign(n_lys_g))*(n_lys_g.astype(str).str.slice(0,0)+", ")
    y=(n_arg_g.astype(str)+"ARG")*np.sign(n_arg_g)+comma+(n_lys_g.astype(str)+"LYS")*np.sign(n_lys_g)
    
    
    dists.loc[:,"G-site"]=y.copy()
    dists.loc[dists["G-site"]=="","G-site"]="NONE"


In [34]:
d_sites_checked=pd.read_csv(os.path.join(ploop_in_dir,sites_checked),sep="\t")

In [44]:
#dists_selected=dists2[dists2.include.isna()].copy()
assign_arg_type(dists_selected, d_sites_checked)
assign_mono_type(dists_selected)
assign_type(dists_selected)
dists_selected.to_csv(os.path.join(ploop_wdir,f"{year}_{spec}_t3_dists_with_type_.tsv"), sep="\t")
dists_selected["G-site"].unique()

array(['NONE', '1ARG', '1LYS', '2LYS', '2ARG', '3ARG', '1ARG, 1LYS'],
      dtype=object)

In [90]:
dists_selected

,superfamily,pfam_acc,pfam_domain,domain_name,Uniprot_Ac,Uniprot_Id,Protein_name_up,Gene_name_up,PDBID,model,nuc_type,nuc_chain,nuc_id,nuc_gamma_moiety_type,nuc_gamma_moiety_chain,nuc_gamma_moiety_id,mg_in_site_4a_from_beta,AG-site,G-site,TYPE_ARG,LYS-site,asn_TYPE,Lys_res_ch,Lys_res_id,nz-alpha-atom,nz-alpha-dist,nz-gamma-atom,nz-gamma-dist,Arg_res_ch,Arg_res_id,nh1-alpha-atom,nh1-alpha-dist,nh1-gamma-atom,nh1-gamma-dist,nh2-alpha-atom,nh2-alpha-dist,nh2-gamma-atom,nh2-gamma-dist,ne-alpha-atom,ne-alpha-dist,ne-gamma-atom,ne-gamma-dist,asn_ID,asn_ne2-alpha-atom,asn_ne2-alpha-dist,asn_ne2-gamma-atom,asn_ne2-gamma-dist,Surr_N_BB,Surr_N_SCh,gly13-chain,gly13-type,gly13-id,nuc-to-g13-atom,dist-gly13,lys-ploop-info,ploopk-dist,resolution,method,pdbname,pdbdate,include,SerK+1-Mg,WB-Asp/Glu,WBD-SerK+1_dist,WBD-Mg,water_present,pfam_comm
0,TRAFAC,PF00063,Myosin_head,Myosin head (motor domain),P10587,MYH11_CHICK,Myosin-11,MYH11,1br1,0,ADP,A,998,ALF,A,999,MG,NaN,NaN,NaN,NaN,,A,250,O1A,8.639585,F4,7.628103,A,247,O2A,10.024054,F3,5.332140,O2A,11.278693,F3,6.160625,O2A,10.615997,F3,6.615000,A242,O2A,2.916646,F2,3.817497,GLY_A468..F1(2.55); SER_A246..F2(2.70),,A,GLY,180,F1,3.655413,A183,2.780986,3.5,x-ray diffraction,SMOOTH MUSCLE MYOSIN MOTOR DOMAIN-ESSENTIAL LI...,1998-08-26,NaN,2.159976,ASP_A465,3.946227,3.889449,True,Ok
1,TRAFAC,PF00063,Myosin_head,Myosin head (motor domain),P10587,MYH11_CHICK,Myosin-11,MYH11,1br1,0,ADP,C,998,ALF,C,999,MG,NaN,NaN,NaN,NaN,,C,250,O1A,8.718588,F4,7.675519,C,247,O2A,9.955953,F3,5.308627,O2A,11.298747,F3,6.233797,O2A,10.617236,F3,6.681140,C242,O2A,2.969201,F2,3.955435,GLY_C468..F1(2.46); SER_C246..F2(2.75),,C,GLY,180,F1,3.738719,C183,2.751563,3.5,x-ray diffraction,SMOOTH MUSCLE MYOSIN MOTOR DOMAIN-ESSENTIAL LI...,1998-08-26,NaN,2.166397,ASP_C465,3.922912,3.763648,True,Ok
2,TRAFAC,PF00063,Myosin_head,Myosin head (motor domain),P10587,MYH11_CHICK,Myosin-11,MYH11,1br1,0,ADP,E,998,ALF,E,999,MG,NaN,NaN,NaN,NaN,,E,250,O1A,8.767497,F4,7.666852,E,247,O2A,10.116271,F3,5.496902,O2A,11.470340,F3,6.434052,O2A,10.821660,F3,6.894290,E242,O2A,3.156481,F2,3.965111,GLY_E468..F1(2.48); SER_E179..F1(3.99); SER_E2...,,E,GLY,180,F1,3.665947,E183,2.710544,3.5,x-ray diffraction,SMOOTH MUSCLE MYOSIN MOTOR DOMAIN-ESSENTIAL LI...,1998-08-26,NaN,2.076092,ASP_E465,3.938586,3.718373,True,Ok
3,TRAFAC,PF00063,Myosin_head,Myosin head (motor domain),P10587,MYH11_CHICK,Myosin-11,MYH11,1br1,0,ADP,G,998,ALF,G,999,MG,NaN,NaN,NaN,NaN,,G,250,O1A,8.785262,F4,7.654045,G,247,O2A,10.132792,F3,5.477052,O2A,11.498631,F3,6.433871,O2A,10.795943,F3,6.833364,G242,O2A,3.129764,F2,3.859841,GLY_G468..F1(2.53); SER_G246..F2(2.78),,G,GLY,180,F1,3.664732,G183,2.774285,3.5,x-ray diffraction,SMOOTH MUSCLE MYOSIN MOTOR DOMAIN-ESSENTIAL LI...,1998-08-26,NaN,2.066263,ASP_G465,3.922849,3.771162,True,Ok
4,TRAFAC,PF00063,Myosin_head,Myosin head (motor domain),P10587,MYH11_CHICK,Myosin-11,MYH11,1br2,0,ADP,A,998,ALF,A,999,MG,NaN,NaN,NaN,NaN,,A,250,O1A,8.386499,F4,7.087786,A,247,O3A,9.803608,F3,4.609398,O3A,12.066981,F3,6.799509,O2A,11.023004,F3,6.514483,A242,O2A,2.941894,F2,3.716788,GLY_A468..F1(2.54); SER_A246..F2(2.58),,A,GLY,180,F3,3.980544,A183,2.752079,2.9,x-ray diffraction,SMOOTH MUSCLE MYOSIN MOTOR DOMAIN COMPLEXED WI...,1998-08-26,NaN,2.013697,ASP_A465,2.795241,3.434139,True,Ok
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3662,TRAFAC,PF02263,GBP,"Guanylate-binding protein, N-terminal domain",Q8WXF7,ATLA1_HUMAN,Atlastin-1,ATL1 GBP3 SPG3A,6b9f,0,GDP,B,506,ALF,B,507,MG,NaN,NaN,NaN,NaN,,B,78,O3A,9.275437,F1,9.620522,B,77,O1A,4.430065,F4,4.947354,O1A,2.835662,F4,2.957734,O3A,4.683665,F1,2.908955,B177,O3A,7.003891,F3,7.956620,GLU_B119..F1(3.91); THR_B120..F2(3.59); GLY_B1...,,B,ARG,77,F1,2.910028,B80,2.570031,1.9,x-ray diffraction,Human ATL1 mutan

In [36]:
dists_selected.to_csv(os.path.join(ploop_wdir,f"{year}_{spec}_dists_with_type.tsv"), sep="\t")

In [72]:
dists_selected[dists_selected.TYPE_ARG=="FORK."].to_csv(os.path.join(ploop_wdir,"forks.tsv"),sep="\t")

In [73]:

d_sites_checked

,PDB,nuc,arg,argtype
0,1kof,B501,B124,NH2 weak*
1,1qhx,A501,A133,NH2 weak*
2,1wq1,R167,G789,FORK*
3,2bc9,A593,A48,FORK*
4,2g83,B355,B178,NH1 weak*
5,2wss,O600,K373,NH1 weak*
6,3adc,B2002,B116,NH2 weak*
7,3add,B2002,B116,FORK*
8,3bb1,H281,G133,FORK*
9,3ex7,C414,C370,FORK*


In [7]:
import pandas as pd
import os

year=2022
spec="Ver5_hy_fallback"
dists_selected=pd.read_csv(os.path.join(ploop_wdir,f"{year}_{spec}_t3_dists_with_type_.tsv"), sep="\t")

In [74]:
fork=dists_selected[dists_selected.TYPE_ARG=="FORK."]

## Select columns for excel

In [18]:
dists_selected.columns

Index(['Unnamed: 0', 'superfamily', 'pfam_acc', 'pfam_domain', 'domain_name',
       'Uniprot_Ac', 'Uniprot_Id', 'Protein_name_up', 'Gene_name_up', 'PDBID',
       'model', 'nuc_type', 'nuc_chain', 'nuc_id', 'nuc_gamma_moiety_type',
       'nuc_gamma_moiety_chain', 'nuc_gamma_moiety_id',
       'mg_in_site_4a_from_beta', 'AG-site', 'G-site', 'TYPE_ARG', 'LYS-site',
       'asn_TYPE', 'Lys_res_ch', 'Lys_res_id', 'nz-alpha-atom',
       'nz-alpha-dist', 'nz-gamma-atom', 'nz-gamma-dist', 'Arg_res_ch',
       'Arg_res_id', 'nh1-alpha-atom', 'nh1-alpha-dist', 'nh1-gamma-atom',
       'nh1-gamma-dist', 'nh2-alpha-atom', 'nh2-alpha-dist', 'nh2-gamma-atom',
       'nh2-gamma-dist', 'ne-alpha-atom', 'ne-alpha-dist', 'ne-gamma-atom',
       'ne-gamma-dist', 'asn_ID', 'asn_ne2-alpha-atom', 'asn_ne2-alpha-dist',
       'asn_ne2-gamma-atom', 'asn_ne2-gamma-dist', 'Surr_N_BB', 'Surr_N_SCh',
       'gly13-chain', 'gly13-type', 'gly13-id', 'nuc-to-g13-atom',
       'dist-gly13', 'lys-ploop-info', 'plo

In [12]:
def format_for_excel(dists_selected, reduce_cols, rename_cols):

    r=dists_selected.copy()
    r["nucleotide_type"]=r.nuc_type

    r.loc[r.nucleotide_type.isin(["ADP","GDP"]),"nucleotide_type"]=r.loc[r.nucleotide_type.isin(["ADP","GDP"]),"nucleotide_type"]+"-"+r.loc[r.nucleotide_type.isin(["ADP","GDP"]),"nuc_gamma_moiety_type"]

    r["nucleotide_id"]=r.nuc_chain+r.nuc_id.astype(str)


    r["lys_id"]=r.Lys_res_ch+r.Lys_res_id.astype(str)

    r["arg_id"]=r.Arg_res_ch+r.Arg_res_id.astype(str)

    r["gly13"]=r['gly13-type']+" "+r['gly13-chain']+r['gly13-id'].astype(str)

    r=r[reduce_cols]
    r.rename(columns=rename_cols)
    return r


In [19]:
reduce_cols=[
  'Unnamed: 0',
  'superfamily',
 'pfam_acc',
 'pfam_domain',

 'Uniprot_Id',
 'Protein_name_up',
 'PDBID',
 'resolution',
 'method',
 "nucleotide_type",
 "nucleotide_id",
  'model',
'mg_in_site_4a_from_beta',
 'AG-site',
    'G-site',
 
 "lys_id",
 'LYS-site',


 'nz-alpha-atom',
 'nz-alpha-dist',
 'nz-gamma-atom',
 'nz-gamma-dist',
 
  'arg_id',
'TYPE_ARG',
 'nh1-alpha-atom',
 'nh1-alpha-dist',
 'nh1-gamma-atom',
 'nh1-gamma-dist',
 'nh2-alpha-atom',
 'nh2-alpha-dist',
 'nh2-gamma-atom',
 'nh2-gamma-dist',
 'ne-alpha-atom',
 'ne-alpha-dist',
 'ne-gamma-atom',
 'ne-gamma-dist',
 
 'asn_ID',
 'asn_TYPE',
 
 'asn_ne2-alpha-atom',
 'asn_ne2-alpha-dist',
 'asn_ne2-gamma-atom',
 'asn_ne2-gamma-dist',
 'Surr_N_BB',
 'Surr_N_SCh',
 'gly13',

 'nuc-to-g13-atom',
 'dist-gly13',
 'lys-ploop-info',
 'ploopk-dist',
 'SerK+1-Mg',
'WB-Asp/Glu',
"WBD-SerK+1_dist",
             
"WBD-Mg",
'is_hydro',
'preceding_res',             
"water_present"            
             
 ]

rename_cols={'Unnamed: 0':"Site_Id",
  'asn_ne2-alpha-atom':'asn_nd2-alpha-atom',
 'asn_ne2-alpha-dist':'asn_nd2-alpha-dist',
 'asn_ne2-gamma-atom':'asn_nd2-gamma-atom',
 'asn_ne2-gamma-dist':'asn_nd2-gamma-dist',}



In [20]:
df_dist_formatted_for_excel=format_for_excel(dists_selected, reduce_cols, rename_cols)

In [21]:
df_dist_formatted_for_excel.to_csv(os.path.join(ploop_wdir,f"{year}_{spec}_dists_FOR_XLSX.tsv"), sep="\t")